# Linear regression
Problem:  
Given dataset $(x_i, y_i)_{i=1}^{M}$  
We assume that $ (x, y) $ pairs were genearted with $ y = a * x + b + \epsilon $  
$ \epsilon $ - Gaussian noise

Find approximation $ \hat{y}=a*x+b $ that minimizes mean square error $L = \frac{1}{m}\sum_{i=1}^{M}(y_i-\hat{y})^2 = \frac{1}{m}\sum_{i=1}^{M}(y_i-(a*x+b))^2$  
We need to find parameters $a, b$

# Generate data

In [ ]:
import numpy as np
n_samples = 100
x = np.linspace(-5, 5, 100) # data
a = 0.5
b = -0.8
y = a * x + b + np.random.randn(n_samples) # target / observations
# Given observed pairs (x, y), we want to find a, b that minimizes mean square error

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(x, y)

In [ ]:
plt.scatter(x, y)
plt.plot(x, x*0 + 0.1)
plt.plot(x, x*1)
plt.plot(x, x*-0.5)

# PyTorch datasets
Some default implementations available: https://pytorch.org/docs/stable/data.html  
For images: https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder  
**Dataset** must implement __getitem__(index) (returns indexed element) and __len__() (returns number of elements)  
We could use **TensorDataset** but let's write it

In [ ]:
import torch
from torch.utils.data import Dataset
class NumpyDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target
        
    def __getitem__(self, index):
        # Anything could go here, e.g. image loading from file or a different structure
        # must return samples and targets/labels (x and y in our case)
        datapoint = self.data[index]
        target = self.target[index]
        return torch.tensor([datapoint]), torch.tensor([target])
    
    def __len__(self):
        return len(self.data)
        # must return number of examples

In [ ]:
# Create and test dataset
dataset = NumpyDataset(x, y)

In [ ]:
dataset[4]

In [ ]:
x[4], y[4]

**DataLoader** objects provide a way to efficiently iterate over datasets https://pytorch.org/docs/stable/data.html

In [ ]:
from torch.utils.data import DataLoader
# Create data loader for dataset in dataloader variable
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

In [ ]:
#We can iterate over the dataset
for xs, ys in dataloader:
    print(xs.size(), ys.size())

## PyTorch models
Pytorch models are defined as **Module** objects that need to have a **forward** method implemented that applies transformations (neural network layers) on data. https://pytorch.org/docs/stable/nn.html

In [ ]:
import torch.nn as nn
class LinearModel(nn.Module):
    
    def __init__(self, input_dim):
        super(LinearModel, self).__init__()
        # initialize a Linear layer
        self.fc = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        # apply linear transformation x * a + b and return the result
        out = self.fc(x)
        return out

In [ ]:
# Create model
model = LinearModel(input_dim=1)

In [ ]:
# Show randomly initialized a, b
model.fc.weight, model.fc.bias

In [ ]:
# Evaluate for randomly initialized model
# save results in `prediction`
model.eval()
with torch.no_grad():
    x_data = np.linspace(-5, 5, 100)
    x_data = torch.tensor(x_data, dtype=torch.float) #(100) -> (100x1)
    x_data = x_data.view(-1, 1)
    predictions = model(x_data) # model.forward(x)

In [ ]:
plt.scatter(x, y)
plt.plot(x, predictions.numpy(), color='red')

In [ ]:
# Train the model
from utils import train_mse
train_mse(dataloader, model, n_epochs=500) ## will be covered in the next class

In [ ]:
# Show A, b after training
model.fc.weight, model.fc.bias

# Use the model for prediction

In [ ]:
model.eval()
with torch.no_grad():
    x_data = np.linspace(-5, 5, 100)
    x_data = torch.tensor(x_data, dtype=torch.float) #  size [100]
    x_data = x_data.view(-1, 1) # size [100, 1] (required)
    predictions = model(x_data)

In [ ]:
plt.scatter(x, y)
plt.plot(x, predictions.numpy(), color='red')

In [ ]:
plt.scatter(x, y)
plt.plot(x, predictions.numpy(), color='red')
plt.plot(x, a*x + b, color='green') # a = 0.5, b = -0.8